In [10]:
from pickle_workaround import pickle_load
from TfidfVectorizer import TfidfVectorizer
from vectorizer import tokenize
from nlp import get_top_10_docs
from topics import search_models, plot_results

import numpy as np
import time
import datetime
import matplotlib.pyplot as plt

In [2]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [3]:
from data import get_query_corpus, get_connection
_ = get_connection("/home/mark/Documents/missionmark/missionmark_db_creds")

Connecting to Postgres database...
 -> Connection successful!


In [4]:
query = """SELECT opportunity_id, program_description
           FROM import.govwin_opportunity
           WHERE program_description LIKE('%HVAC%')
        """
doc_ids, corpus = get_query_corpus(query, True)

Loading corpus...
 -> 264 documents loaded!


In [5]:
tfidf = TfidfVectorizer(pickle_load("../output/CountVectorizer.pkl"), pickle_load("../output/TfidfTransformer.pkl"))
tfidf_corpus = tfidf.transform(corpus)

nmf = pickle_load("../output/NMF.pkl")
H = nmf.components_
W = nmf.transform(tfidf_corpus)
W_max = W.max()

Loading ../output/CountVectorizer.pkl from cache...
 -> ../output/CountVectorizer.pkl loaded!
Loading ../output/TfidfTransformer.pkl from cache...
 -> ../output/TfidfTransformer.pkl loaded!
Loading ../output/NMF.pkl from cache...                                         
 -> ../output/NMF.pkl loaded!


In [ ]:
hvac_i = tfidf.vocabulary_["hvac"]
n_hvac_topics = np.count_nonzero(H[:, hvac_i])
hvac_topics = np.argsort(H[:, hvac_i])[::-1][:n_hvac_topics]
hvac_topics_strength = H[:, hvac_i].flatten()[hvac_topics]
print(np.stack(hvac_topics, hvac_topics_strength))

In [ ]:
min_topics = 5
max_topics = 100

time_start = time.time()
costs, intertopic_similarities, interdocument_similarities = search_models(tfidf_corpus, min_topics, max_topics)
time_dif = datetime.timedelta(seconds=round(time.time() - time_start))
_ = plot_results(min_topics, costs, intertopic_similarities, interdocument_similarities, time_dif, tfidf_corpus.shape)

Building NMF topics...
[------------------------------>                   ]  60%        (58/96) 258 iterations